In [1]:
import numpy as np
from numpy.random import uniform 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

**functions to generate SU(3) matrices**

In [2]:

def generate_hermitean():
    H=np.zeros((3,3),np.complex_)
    for i in range(3):
        for j in range(3):
            H[i][j]= uniform(-1,1) + 1j*uniform(-1,1)
    H = (1/2)*(H+np.matrix.getH(H))
    return H

In [3]:
def unitarization(H):
    M=np.zeros((3,3),np.complex_)
   
    for n in range(30):
        M+=(((1j*eps)**n)/(np.math.factorial(n))*np.linalg.matrix_power(H,n))
    detM=np.linalg.det(M)**(1/3)
    M=M/detM
    
    return M

In [4]:
def generate_pool(MatList,Nmatrix):
    
    H=np.zeros((3,3),np.complex_)
       
    for i in range(Nmatrix):
        H=generate_hermitean()
        MatList[i]=unitarization(H)
        MatList[Nmatrix+i]=np.matrix.getH(unitarization(H))
        
    return MatList

**lattice generation function**

generate lattice and 4 3x3 identity matrices on each site

In [5]:
def create_lattice(Npoints,dim):
    points = np.empty( (Npoints,Npoints,Npoints,Npoints,dim,3,3), np.complex_ )
    
    #print(lattice)
    for t in range(Npoints):
        for x in range(Npoints):
            for y in range(Npoints):
                for z in range(Npoints):
                    for mu in range(dim):
                        points[t,x,y,z,mu]=np.identity(3,np.complex_)    
    print("finish lattice creation")  #debug
    return points

**coordinate functions**

moving a given point in + direction or - direction

In [6]:
#moving a coordinate up
def up(coordinate,direction,dimensions,N_points):
    coord_up=np.zeros((dimensions),np.int_)
    up=np.zeros((dimensions),np.int_)
    up[direction]=1
    coord_up=(coordinate + up)%N_points
    return coord_up


#moving a coordinate down
def down(coordinate,direction,dimensions,N_points):
    coord_down=np.zeros((dimensions),np.int_)
    down=np.zeros((dimensions),np.int_)
    down[direction]=1
    coord_down=(coordinate-down)%N_points
    return coord_down

**plaquette**

only up and down 1x1 squares

In [7]:
def plaquette(lattice,mu,x,y,z,t):
    
    staple=0. 
    staple_up=0.
    staple_down=0.
    
    point=[x,y,z,t]
    #point(x+mu)
    point_umu=np.zeros((4),np.int_)
    point_umu=up(point,mu,4,N)
    
    for nu in range(4):
        if nu!=mu :
            
            #point(x+nu)
            point_unu=np.zeros((4),np.int_)
            point_unu=up(point,nu,4,N)
            
            #point(x-nu)
            point_dnu=np.zeros((4),np.int_)
            point_dnu=down(point,nu,4,N)
            
            #point(x+mu+nu)
            point_umu_unu=np.zeros((4),np.int_)
            point_umu_unu=up(point_umu,nu,4,N)
            
            #point(x+mu-nu)
            point_umu_dnu=np.zeros((4),np.int_)
            point_umu_dnu=up(point_dnu,mu,4,N)
            
            U_nu_upmu=np.zeros((3,3),np.complex_)
            U_nu=np.zeros((3,3),np.complex_)
            U_nu_downnu=np.zeros((3,3),np.complex_)
            U_mu_downnu=np.zeros((3,3),np.complex_) 
            U_mu_upmudownnu=np.zeros((3,3),np.complex_)
            
            U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
            U_mu_upnu=lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],mu]
            U_nu=lattice[point[0],point[1],point[2],point[3],nu]
            U_nu_downnu=lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],nu]
            U_mu_downnu=lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],mu]
            U_nu_upmudownnu=lattice[point_umu_dnu[0],point_umu_dnu[1],point_umu_dnu[2],point_umu_dnu[3],nu]
            
            staple_up=np.dot(np.dot(U_nu_upmu,np.matrix.getH(U_mu_upnu)),np.matrix.getH(U_nu))
            staple_down=np.dot(np.dot(np.matrix.getH(U_nu_upmudownnu),np.matrix.getH(U_mu_downnu)),U_nu_downnu)
            
            staple=staple + staple_up + staple_down
                        

    return staple.copy()

**plaquette improved**

considering also rectangles

In [8]:
def plaquette_improved(U,mu,x,y,z,t):
    
    staple_rect=0. 
    
    #point(x)
    point=[x,y,z,t]
    #point(x+mu)
    point_umu=np.zeros((4),np.int_)
    point_umu=up(point,mu,4,N)
    #point(x-mu)
    point_dmu=np.zeros((4),np.int_)
    point_dmu=down(point,mu,4,N)
    #point(x+2mu)
    point_uumu=np.zeros((4),np.int_)
    point_uumu=up(point_umu,mu,4,N)
    for nu in range(4):
        if nu!=mu :
            #point(x+nu)
            point_unu=np.zeros((4),np.int_)
            point_unu=up(point,nu,4,N)
        
            #point(x-nu)
            point_dnu=np.zeros((4),np.int_)
            point_dnu=down(point,nu,4,N)
            
            #point(x+nu+mu)
            point_umu_unu=np.zeros((4),np.int_)
            point_umu_unu=up(point_umu,nu,4,N)
            
            #point(x+mu-nu)
            point_umu_dnu=np.zeros((4),np.int_)
            point_umu_dnu=down(point_umu,nu,4,N)
            
            #point(x+2nu)
            point_uunu=np.zeros((4),np.int_)
            point_uunu=up(point_unu,nu,4,N)
            
            #point(x-2nu)
            point_ddnu=np.zeros((4),np.int_)
            point_ddnu=down(point_dnu,nu,4,N)
            
            #point(x+2mu-nu)
            point_uumu_dnu=np.zeros((4),np.int_)
            point_uumu_dnu=down(point_uumu,nu,4,N)
            
            #point(x+mu-2nu)
            point_umu_ddnu=np.zeros((4),np.int_)
            point_umu_ddnu=down(point_umu_dnu,nu,4,N)
            
            #point(x-mu-nu)
            point_dmu_dnu=np.zeros((4),np.int_)
            point_dmu_dnu=down(point_dmu,nu,4,N)
            
            #point(x-mu+nu)
            point_dmu_unu=np.zeros((4),np.int_)
            point_dmu_unu=up(point_dmu,nu,4,N)
            
            U_mu_upmu=np.zeros((3,3),np.complex_)
            U_nu_upupmu=np.zeros((3,3),np.complex_)
            U_mu_upmuupnu=np.zeros((3,3),np.complex_)
            U_mu_upnu=np.zeros((3,3),np.complex_)
            U_nu=np.zeros((3,3),np.complex_)
            U_nu_upupmudownnu=np.zeros((3,3),np.complex_)
            U_mu_upmudownnu=np.zeros((3,3),np.complex_)
            U_mu_downnu=np.zeros((3,3),np.complex_)
            U_nu_downnu=((3,3),np.complex_)
            U_nu_upmu=np.zeros((3,3),np.complex_)
            U_nu_upmuupnu=((3,3),np.complex_)
            U_mu_upupnu=np.zeros((3,3),np.complex_)
            U_nu_upnu=((3,3),np.complex_)
            U_nu_upmudownnu=np.zeros((3,3),np.complex_)
            U_nu_upmudowndownnu=np.zeros((3,3),np.complex_)
            U_mu_downmuupnu=np.zeros((3,3),np.complex_)
            U_mu_downmudownnu=np.zeros((3,3),np.complex_)
            U_nu_downmudownnu=np.zeros((3,3),np.complex_)
            U_mu_downmu=np.zeros((3,3),np.complex_)
            U_nu_downmu=np.zeros((3,3),np.complex_)
            U_nu_downdownnu=np.zeros((3,3),np.complex_)
            U_mu_downdownnu=np.zeros((3,3),np.complex_)
            
            
            U_mu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],mu]
            U_nu_upupmu=lattice[point_uumu[0],point_uumu[1],point_uumu[2],point_uumu[3],nu]
            U_mu_upmuupnu=lattice[point_umu_unu[0],point_umu_unu[1],point_umu_unu[2],point_umu_unu[3],mu]
            U_mu_upnu=lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],mu]
            U_nu=lattice[point[0],point[1],point[2],point[3],nu]
            U_nu_upupmudownnu=lattice[point_uumu_dnu[0],point_uumu_dnu[1],point_uumu_dnu[2],point_uumu_dnu[3],nu]
            U_mu_upmudownnu=lattice[point_umu_dnu[0],point_umu_dnu[1],point_umu_dnu[2],point_umu_dnu[3],mu]
            U_mu_downnu=lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],mu]
            U_nu_downnu=lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],nu]
            U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
            U_nu_upmuupnu=lattice[point_umu_unu[0],point_umu_unu[1],point_umu_unu[2],point_umu_unu[3],nu]
            U_mu_upupnu=lattice[point_uunu[0],point_uunu[1],point_uunu[2],point_uunu[3],mu]
            U_nu_upnu=lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],nu]
            U_nu_upmudownnu=lattice[point_umu_dnu[0],point_umu_dnu[1],point_umu_dnu[2],point_umu_dnu[3],nu]
            U_nu_upmudowndownnu=lattice[point_umu_ddnu[0],point_umu_ddnu[1],point_umu_ddnu[2],point_umu_ddnu[3],nu]
            U_mu_downmuupnu=lattice[point_dmu_unu[0],point_dmu_unu[1],point_dmu_unu[2],point_dmu_unu[3],mu]
            U_mu_downmudownnu=lattice[point_dmu_dnu[0],point_dmu_dnu[1],point_dmu_dnu[2],point_dmu_dnu[3],mu]
            U_nu_downmudownnu=lattice[point_dmu_dnu[0],point_dmu_dnu[1],point_dmu_dnu[2],point_dmu_dnu[3],nu]
            U_nu_downmu=lattice[point_dmu[0],point_dmu[1],point_dmu[2],point_dmu[3],nu]
            U_mu_downmu=lattice[point_dmu[0],point_dmu[1],point_dmu[2],point_dmu[3],mu]
            U_nu_downdownnu=U[point_ddnu[0],point_ddnu[1],point_ddnu[2],point_ddnu[3],nu]
            U_mu_downdownnu=U[point_ddnu[0],point_ddnu[1],point_ddnu[2],point_ddnu[3],mu]
            
            
            staple_rect+=np.dot(np.dot(U_mu_upmu,U_nu_upupmu),np.dot(np.matrix.getH(U_mu_upmuupnu),np.dot(np.matrix.getH(U_mu_upnu),np.matrix.getH(U_nu)))) \
                                +np.dot(np.dot(U_mu_upmu,np.matrix.getH(U_nu_upupmudownnu)),np.dot(np.matrix.getH(U_mu_upmudownnu),np.dot(np.matrix.getH(U_mu_downnu),U_nu_downnu))) \
                                +np.dot(np.dot(U_nu_upmu,U_nu_upmuupnu),np.dot(np.matrix.getH(U_mu_upupnu),np.dot(np.matrix.getH(U_nu_upnu),np.matrix.getH(U_nu)))) \
                                +np.dot(np.dot(np.matrix.getH(U_nu_upmudownnu),np.matrix.getH(U_nu_upmudowndownnu)),np.dot(np.matrix.getH(U_mu_downdownnu),np.dot(U_nu_downdownnu,U_nu_downnu))) \
                                +np.dot(np.dot(U_nu_upmu,np.matrix.getH(U_mu_upnu)),np.dot(np.matrix.getH(U_mu_downmuupnu),np.dot(np.matrix.getH(U_nu_downmu),U_mu_downmu))) \
                                +np.dot(np.dot(np.matrix.getH(U_nu_upmudownnu),np.matrix.getH(U_mu_downnu)),np.dot(np.matrix.getH(U_mu_downmudownnu),np.dot(U_nu_downmudownnu,U_mu_downmu)))
    return staple_rect.copy()

**metropolis function**

In [9]:
def lattice_update(lattice,Xs):
    
    n_hits=10
    gamma=0
    gamma_imp=0
    
    for x in range(N):
        for y in range(N):
            for z in range(N):
                for t in range(N):
                    for mu in range(4):
                        
                        coord=[x,y,z,t]
                        gamma=plaquette(lattice,mu,x,y,z,t)
                        gamma_imp=plaquette_improved(lattice,mu,x,y,z,t) 
                        
                        for i in range(n_hits):#do n_hits of metropolis update
                            
                            xi=np.random.randint(2, N_matrix*2)
                            U_old=lattice[coord[0],coord[1],coord[2],coord[3],mu]
                            U_new=np.dot(Xs[xi],U_old)
                            
                            dS = -beta_imp/(3)*(5/(3*u0**4)*np.real(np.trace(np.dot((U_new-U_old),gamma)))-1/(12*u0**6)*np.real(np.trace(np.dot((U_new-U_old),gamma_imp)))) # change in the improved action

                            if dS<0 or np.exp(-dS)>np.random.uniform(0,1):
                                lattice[coord[0],coord[1],coord[2],coord[3],mu] = U_new

**wilson axa and ax2a**

given a point in the lattice

In [10]:
def wilson_axa(lattice,x,y,z,t):

    w_axa=0
    
    point_umu=np.zeros((dim),np.int_)
    point_unu=np.zeros((dim),np.int_)
    point=[x,y,z,t]
    
    U_mu=np.zeros((3,3),np.complex_)
    U_nu_upmu=np.zeros((3,3),np.complex_)
    U_dag_mu_upnu=np.zeros((3,3),np.complex_)
    U_dag_nu=np.zeros((3,3),np.complex_)
    
    for mu in range(4):
        
        #point(x+mu)
        point_umu= up(point,mu,dim,N)
        for nu in range(mu):
            
            #point(x+nu)
            point_unu= up(point,nu,dim,N)
            
            
            U_mu=lattice[point[0],point[1],point[2],point[3],mu]
            U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
            U_dag_mu_upnu=np.matrix.getH(lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],mu])
            U_dag_nu=np.matrix.getH(lattice[point[0],point[1],point[2],point[3],nu])
            
            w_axa += np.trace( np.dot(U_mu,np.dot(np.dot(U_nu_upmu,U_dag_mu_upnu), U_dag_nu)))
            
    return np.real(w_axa)/(3.*6.)


In [11]:
def wilson_ax2a(lattice,x,y,z,t):

    w_ax2a = 0.
    point=[x,y,z,t]
    
    U_mu=np.zeros((3,3),np.complex_)
    U_nu_upnu=np.zeros((3,3),np.complex_)
    U_nu_upmu=np.zeros((3,3),np.complex_)
    U_mu_upupnu=np.zeros((3,3),np.complex_)
    U_nu_upmuupnu=np.zeros((3,3),np.complex_)
    U_nu=np.zeros((3,3),np.complex_)
    
    for mu in range(4):
        point_umu=np.zeros((4),np.int_)
        point_umu=up(point,mu,4,N)
        for nu in range(3,mu,-1):
            if nu!=mu :
                
                #next site on nu
                point_unu=np.zeros((4),np.int_)
                point_unu=up(point,nu,4,N)
                
                #next site on nu and mu
                point_umu_unu=np.zeros((4),np.int_)
                point_umu_unu=up(point_unu,mu,4,N)
                
                #next site on 2nu
                point_uunu=np.zeros((4),np.int_)
                point_uunu=up(point_unu,nu,4,N)
                
                U_mu=lattice[point[0],point[1],point[2],point[3],mu]
                U_nu_upmuupnu=lattice[point_umu_unu[0],point_umu_unu[1],point_umu_unu[2],point_umu_unu[3],nu]
                U_nu_upnu=lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],nu]
                U_mu_upupnu=lattice[point_uunu[0],point_uunu[1],point_uunu[2],point_uunu[3],mu]
                U_nu=lattice[point[0],point[1],point[2],point[3],nu]
                U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
                
                w_ax2a += np.trace(np.dot(U_mu,\
                        np.dot(np.dot(U_nu_upmu,U_nu_upmuupnu),\
                        np.dot(np.matrix.getH(U_mu_upupnu),\
                        np.dot(np.matrix.getH(U_nu_upnu),np.matrix.getH(U_nu))))))

        
    return np.real(w_ax2a)/(3.*6.)

functions to run over the whole lattice and computing wilson loops axa and ax2a

In [12]:
def compute_wilson_axa(lattice,Xs,WL_axa,N_cf,N_cor):
    
    for alpha in range(0,N_cf): 
        for j in range(0,N_cor):
            lattice_update(lattice,Xs)
        for x in range(0,N):
            for y in range(0,N):
                for z in range(0,N):
                    for t in range(0,N):
                        WL_axa[alpha]+=wilson_axa(lattice,x,y,z,t) 
        WL_axa[alpha]=WL_axa[alpha]/N**4
        print(alpha+1,WL_axa[alpha])

In [13]:
def compute_wilson_ax2a(lattice,Xs,WL_ax2a,N_cf,N_cor):
    
    for alpha in range(N_cf): 
        for j in range(N_cor):
            lattice_update(lattice,Xs)
        for x in range(N):
            for y in range(N):
                for z in range(N):
                    for t in range(N):
                        WL_ax2a[alpha]+=wilson_ax2a(lattice,x,y,z,t) 
        WL_ax2a[alpha]=WL_ax2a[alpha]/N**4
        print(alpha+1,WL_ax2a[alpha])

**main**

input parameters, allocation of arrays

In [14]:
a=.25
N = 8
N_cf=10
N_matrix=100
N_cor = 50
beta = 5.5
beta_imp=1.719
u0=0.797
dim=4
eps=0.24

lattice=np.zeros((N,N,N,N,4,3,3),np.complex_)
WL_axa=np.zeros((N_cf),np.double)
WL_ax2a=np.zeros((N_cf),np.double)
update_Ms=np.zeros((N_matrix*2,3,3),np.complex_)

**thermalization**

In [15]:
lattice=create_lattice(N,dim) #creating lattice
update_Ms=generate_pool(update_Ms,N_matrix)#generating the list of update_matrices 

for i in tqdm(range(2*N_cor)):
    lattice_update(lattice,update_Ms)
    
print("thermalization done")

  0%|          | 0/100 [00:00<?, ?it/s]

finish lattice creation


100%|██████████| 100/100 [31:03<00:00, 18.64s/it]

thermalization done


**computation**

of Wilson loops axa and ax2a for N_cf configurations and printing

In [16]:
compute_wilson_axa(lattice,update_Ms,WL_axa,N_cf,N_cor)

1 0.5393135532528625
2 0.5388719714311253
3 0.5374799950236901
4 0.5356847795129683
5 0.540397739509551
6 0.5440268775498887
7 0.5429209046404438
8 0.539394660242272
9 0.5435974652985989
10 0.5407131261791879


In [17]:
compute_wilson_ax2a(lattice,update_Ms,WL_ax2a,N_cf,N_cor)

1 0.2855341614244994
2 0.2782047335791637
3 0.2851623265351508
4 0.2870598162485029
5 0.28303133185888385
6 0.2805393593070499
7 0.2883435839969175
8 0.2772165417795397
9 0.28525295789145577
10 0.283864025082817


**averages**

and error analysis

In [19]:
def avg_analysis(wl):
    wl_avg=0.
    wl_sq_avg=0.
    wl_dev_std=0.
    
    for alpha in range(N_cf):
        wl_avg += wl[alpha]
        wl_sq_avg+=wl[alpha]**2 
        
    wl_avg=wl_avg/N_cf
    wl_sq_avg=wl_sq_avg/N_cf
    wl_dev_std=(abs(wl_sq_avg-wl_avg**2)/N_cf)**(1/2)
    return wl_avg,wl_dev_std

In [20]:
WL_axa_avg = 0.
WL_axa_dev_std=0.
WL_ax2a_avg = 0.
WL_a2xa_dev_std=0.

In [21]:
WL_axa_avg,WL_axa_dev_std=avg_analysis(WL_axa)
WL_ax2a_avg,WL_ax2a_dev_std=avg_analysis(WL_ax2a)

**file writing**

In [22]:
file1 = open("wilson_loop_imp.txt", "w")
file1.write('######################################\n')
file1.write(' Wilson loop axa action improved\n')

33

In [23]:
for alpha in range(N_cf):
    content = str(WL_axa[alpha])
    file1.write(str(alpha)+'  '+content+'\n')
file1.write('\n')
file1.write('mean:     '+str(WL_axa_avg)+ '    +/- '+str(WL_axa_dev_std))

57

In [24]:
file1.write('\n######################################\n')

40

In [25]:
file1.write('\n Wilson loop ax2a action improved\n')
for alpha in range(N_cf):
    content = str(WL_ax2a[alpha])
    file1.write(str(alpha)+'  '+content+'\n')
file1.write('\n')
file1.write('mean:     '+str(WL_ax2a_avg)+ '    +/- '+str(WL_ax2a_dev_std))
file1.write('\n######################################\n')
file1.close()

check file writing

In [26]:
file1 = open("wilson_loop_imp.txt", "r")
content = file1.read()
 
print("\nContent:\n", content)
file1.close()


Content:
 ######################################
 Wilson loop axa action improved
0  0.5393135532528625
1  0.5388719714311253
2  0.5374799950236901
3  0.5356847795129683
4  0.540397739509551
5  0.5440268775498887
6  0.5429209046404438
7  0.539394660242272
8  0.5435974652985989
9  0.5407131261791879

mean:     0.5402401072640589    +/- 0.0008053636490438512
######################################

 Wilson loop ax2a action improved
0  0.2855341614244994
1  0.2782047335791637
2  0.2851623265351508
3  0.2870598162485029
4  0.28303133185888385
5  0.2805393593070499
6  0.2883435839969175
7  0.2772165417795397
8  0.28525295789145577
9  0.283864025082817

mean:     0.2834208837703981    +/- 0.0011076247493438357
######################################

